## Seed Genernator

In [ ]:
# import module
import random
import datetime

def seed_generator() -> list:
    seed_lst = []
    for i in range(0, 11):
        ct = datetime.datetime.now()
        random.seed(a=ct.timestamp(), version=2)
        seed_lst.append('{0:04d}'.format(int(random.random()*10000)))

    return seed_lst


----

## Config File Generator

In [ ]:
# config text

def config_generator(*argv) -> str:
    seed_lst = argv[0]
    for seed in seed_lst:
        config_str = '''
        {
        "model_name_or_path": "tartuNLP/EstBERT",
        "cache_dir": "/gpfs/space/home/chenghan/ner/models",
        "dataset_name": "estner",
        "max_seq_length": 128,
        "do_train": "True",
        "do_eval": "True",
        "do_predict": "True",
        "output_dir": "/gpfs/space/home/chenghan/ner/output/new_ner/lr3e5s{0}",
        "overwrite_output_dir": "True",
        "report_to": "wandb",
        "evaluation_strategy": "epoch",
        "per_device_train_batch_size": 16,
        "per_device_eval_batch_size": 16,
        "learning_rate": 3e-5,
        "weight_decay": 0.01,
        "adam_beta1": 0.9,
        "adam_beta2": 0.98,
        "adam_epsilon": 1e-6,
        "num_train_epochs": 150,
        "lr_scheduler_type": "polynomial",
        "logging_strategy": "epoch",
        "logging_first_step": "True",
        "logging_steps": 150,
        "save_strategy": "epoch",
        "seed": {1},
        "fp16": "True",
        "fp16_full_eval": "False",
        "run_name": "ner_estbert_new_b16_lr3e-5_s{2}",
        "load_best_model_at_end": "True",
        "metric_for_best_model": "eval_f1",
        "return_tag_level_metrics": "False",
        "save_total_limit": 3
        }'''.format(seed,seed,seed)
    return config_str


--------
## Modify token

In [6]:
import json

_PATH = r'./data/EstNER_token'
file_list = {'EstNER_token_test.json',
             'EstNER_token_dev.json', 'EstNER_token_train.json'}
entity_list = {'PER', 'GPE', 'LOC', 'ORG'}
cnt=0
for jf in file_list:
    with open(f'{_PATH}/{jf}', 'r+') as f:
        data = json.load(f)
        for doc in data['documents']:
            for sent in doc['sentences']:
                for word in sent['words']:
                    if (any(map(word['ner_1'].__contains__, entity_list))):
                        pass
                    else:
                        word['ner_1'] = "O" # trandfer entities to un-labled

        f.seek(0)  # rewind
        json.dump(data, f)
        f.truncate()


In [3]:
import pandas as pd
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("d3auuv/huggingface")

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k, v in run.config.items()
         if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
})

_JSON_FILE_NAME = 'project.json'
runs_df.to_json(_JSON_FILE_NAME)


In [18]:
import json
from openpyxl import workbook,load_workbook

ent_lst=['PER','GPE','LOC','ORG','PROD','EVENT','DATE','TIME','TITLE','MONEY','PERCENT']
score_lst=['f1','recall','precision']
with open(_JSON_FILE_NAME, 'r') as f:
    data = json.load(f)

for i in range(11):
    sum = data['summary']
    model = sum[str(i)]
    print(data['config'][str(i)]['seed'])
    for key,value in model.items():
        if key.find(f'eval/TITLE') != -1:
            print(key,value)

7849
eval/TITLE_precision 0.7777777778
eval/TITLE_number 20
eval/TITLE_f1 0.7368421053
eval/TITLE_recall 0.7
6891
eval/TITLE_recall 0.75
eval/TITLE_f1 0.7692307692
eval/TITLE_precision 0.7894736842
eval/TITLE_number 20
9470
eval/TITLE_precision 0.7777777778
eval/TITLE_f1 0.7368421053
eval/TITLE_recall 0.7
eval/TITLE_number 20
6523
eval/TITLE_recall 0.8
eval/TITLE_f1 0.7441860465
eval/TITLE_precision 0.6956521739
eval/TITLE_number 20
6730
eval/TITLE_f1 0.7567567568
eval/TITLE_number 20
eval/TITLE_recall 0.7
eval/TITLE_precision 0.8235294118
5490
eval/TITLE_number 20
eval/TITLE_recall 0.7
eval/TITLE_f1 0.7
eval/TITLE_precision 0.7
3552
eval/TITLE_number 20
eval/TITLE_recall 0.7
eval/TITLE_precision 0.7368421053
eval/TITLE_f1 0.7179487179
4378
eval/TITLE_f1 0.6666666667
eval/TITLE_recall 0.65
eval/TITLE_precision 0.6842105263
eval/TITLE_number 20
5436
eval/TITLE_number 20
eval/TITLE_recall 0.7
eval/TITLE_f1 0.7368421053
eval/TITLE_precision 0.7777777778
1024
eval/TITLE_number 20
eval/TITL